In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///resources/Chinook.db")

In [3]:
from langchain.tools import tool

@tool
def sql_query(query: str) -> str:

    """Obtain information from the database using SQL queries"""

    try:
        return db.run(query)
    except Exception as e:
        return f"Error: {e}"

sql_query.invoke("SELECT * FROM Artist LIMIT 10")

/Users/sean/Documents/lca/demo/lc-foundations/.venv/lib/python3.12/site-packages/pydantic/v1/main.py:1054: UserWarning: LangSmith now uses UUID v7 for run and trace identifiers. This warning appears when passing custom IDs. Please use: from langsmith import uuid7
            id = uuid7()
Future versions will require UUID v7.
  input_data = validator(cls_, input_data)


"[(1, 'AC/DC'), (2, 'Accept'), (3, 'Aerosmith'), (4, 'Alanis Morissette'), (5, 'Alice In Chains'), (6, 'Antônio Carlos Jobim'), (7, 'Apocalyptica'), (8, 'Audioslave'), (9, 'BackBeat'), (10, 'Billy Cobham')]"

In [4]:
from langchain.agents import create_agent

agent = create_agent(
    model="gpt-5-nano",
    tools=[sql_query]
)

In [11]:
from langchain.messages import HumanMessage

question = HumanMessage(content="Who is the most popular artist beginning with 'S' in this database?")

response = agent.invoke(
    {"messages": [question]}
)

In [12]:
from pprint import pprint

pprint(response['messages'])

[HumanMessage(content="Who is the most popular artist beginning with 'S' in this database?", additional_kwargs={}, response_metadata={}, id='7f19fc7c-e939-4df7-aa66-0604930bd0be'),
 AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 1320, 'prompt_tokens': 142, 'total_tokens': 1462, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 1280, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CfZnMzhYnFzfHmWBOy2xWfb0KXKGi', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--6cb3af4f-2a60-41cc-ae61-b7583a1797f0-0', tool_calls=[{'name': 'sql_query', 'args': {'query': "SELECT name FROM artists WHERE name LIKE 'S%' ORDER BY popularity DESC LIMIT 1;"}, 'id': 'call_3NANQLD4FWK3AiGSEQCNPDB

In [16]:
print(response["messages"][-3].tool_calls[0]['args']['query'])

SELECT s.Name FROM (SELECT a.ArtistId, a.Name, SUM(il.Quantity * il.UnitPrice) AS total_sales
FROM Artist a
JOIN Album al ON al.ArtistId = a.ArtistId
JOIN Track t ON t.AlbumId = al.AlbumId
JOIN InvoiceLine il ON il.TrackId = t.TrackId
GROUP BY a.ArtistId, a.Name) AS s
WHERE s.Name LIKE 'S%'
ORDER BY s.total_sales DESC
LIMIT 1;
